# Import

In [49]:
import numpy as np
import random
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import f1_score
from scipy.stats import mode

import re
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

from cleanlab import Datalab

# seed 고정

In [50]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정


# 데이터 로드

In [51]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# 전처리

## 이상치 처리

In [52]:
# '부채_대비_소득_비율'이 5000보다 큰 행의 인덱스를 찾기
index_to_drop = train_df[train_df['부채_대비_소득_비율'] > 5000].index

# 찾은 인덱스를 사용하여 해당 행을 삭제
train_df.drop(index_to_drop, inplace=True)

# 삭제한 행으로 인해 인덱스를 리셋
train_df.reset_index(drop=True, inplace=True)

## 피처 엔지니어링1 (피처 생성)

In [53]:
# '총상환원금비율', '총상환이자비율' 피처 생성
train_df['총상환원금비율'] = (train_df['총상환원금']/train_df['대출금액']) * 100
test_df['총상환원금비율'] = (test_df['총상환원금']/test_df['대출금액']) * 100
train_df['총상환이자비율'] = (train_df['총상환이자']/train_df['대출금액']) * 100
test_df['총상환이자비율'] = (test_df['총상환이자']/test_df['대출금액']) * 100

# '연간소득/대출금액' 피처 생성
train_df['연간소득/대출금액'] = train_df['연간소득']/train_df['대출금액']
test_df['연간소득/대출금액'] = test_df['연간소득']/test_df['대출금액']


# '총상환이자', '총상환원금' 피처 drop
train_df = train_df.drop(columns=['총상환이자', '총상환원금'])
test_df = test_df.drop(columns=['총상환이자', '총상환원금'])

- **총상환원금비율, 총상환이자비율:** '대출금액'이 많을수록 '총상환원금'과 '총상환이자'는 당연히 수가 클거라 판단 후 '총상환원금', '총상환이자'를 '대출금액'으로 나누어서 비율로 보기로 결정


- **연간소득/대출금액:** 대출금액에 따른 연간소득을 살펴보았을 때 본인의 소득 수준에서 대출을 얼마나 하는지 알 수 있을 것이라 판단 후 '연간소득'을 '대출금액'으로 나누어서 새 피처를 생성


- **drop:** 피처 생성 후 더이상 필요 없을 것 같은 피처 drop ('총상환이자', '총상환원금')

## 스케일링

In [54]:
# 로그변환
columns_to_transform = ['연간소득', '부채_대비_소득_비율', '총상환원금비율', '총상환이자비율', '총연체금액']

train_df[columns_to_transform] = train_df[columns_to_transform].apply(lambda x: np.log1p(x))
test_df[columns_to_transform] = test_df[columns_to_transform].apply(lambda x: np.log1p(x))

- skewness가 큰 데이터셋이기 때문에 로그 스케일링 진행

## 피처 엔지니어링2 (피처 생성)

In [55]:
# '연체계좌수/총계좌수', '연간소득/총계좌수' 피처 생성
train_df['연체계좌수/총계좌수'] = (train_df['연체계좌수']/train_df['총계좌수']) * 100
test_df['연체계좌수/총계좌수'] = (test_df['연체계좌수']/test_df['총계좌수']) * 100
train_df['연간소득/총계좌수'] = (train_df['연간소득']/train_df['총계좌수']) * 100
test_df['연간소득/총계좌수'] = (test_df['연간소득']/test_df['총계좌수']) * 100

# '연체계좌수', '총계좌수', 피처 drop
train_df = train_df.drop(columns=['연체계좌수', '총계좌수'])
test_df = test_df.drop(columns=['연체계좌수', '총계좌수'])

- ***연간소득 스케일링 진행 후 피처 생성***


- **연체계좌수/총계좌수:** 총계좌 중 얼마나 많은 계좌가 연체되었는지의 정보를 통해 대출등급을 추정할 수 있을 것이라 판단 후 '연체계좌수'를 '총계좌수'로 나누어서 새 피처를 생성


- **연간소득/총계좌수:** 소득에 따라 계좌를 얼마나 가지고 있는지의 정보를 통해 대출등급을 추정할 수 있을 것이라 판단 후 (돈은 없고 계좌만 많은 사람 등등) '연간소득'를 '총계좌수'로 나누어서 새 피처를 생성


- **drop:** 피처 생성 후 더이상 필요 없을 것 같은 피처 drop ('연체계좌수', '총계좌수')

## 범주형 데이터 처리

### 근로기간 0~10 범주로 바꿔주기

In [56]:
def clean_work_experience(value):
    if value == 'Unknown':
        return None
    elif '< 1 year' in value or '<1 year' in value:
        return '0'
    else:
        match = re.search(r'\d+', str(value))
        return match.group() if match else None

# Apply the cleaning function to '근로기간' column
train_df['근로기간'] = train_df['근로기간'].apply(clean_work_experience)
test_df['근로기간'] = test_df['근로기간'].apply(clean_work_experience)

# Convert the column to numeric type
train_df['근로기간'] = pd.to_numeric(train_df['근로기간'], errors='coerce')
test_df['근로기간'] = pd.to_numeric(test_df['근로기간'], errors='coerce')

# 결측치 처리
train_df['근로기간'].fillna(round(train_df['근로기간'].mean()), inplace=True)
test_df['근로기간'].fillna(round(train_df['근로기간'].mean()), inplace=True)

### 주택소유상태의 ANY를 MORTAGE(최빈값)으로 변환

In [57]:
train_df['주택소유상태'] = train_df['주택소유상태'].replace({'ANY':'MORTGAGE'})
test_df['주택소유상태'] = test_df['주택소유상태'].replace({'ANY':'MORTGAGE'})

### 대출목적의 범주를 '부채통합',' '주택', '신용카드', '기타'의 4개의 범주로 변환

In [58]:
train_df['대출목적'] = train_df['대출목적'].replace({'이사':'주택', '주택개선':'주택'})
train_df.loc[~train_df['대출목적'].isin(['주택', '부채 통합', '신용 카드']), '대출목적'] = '기타'
test_df['대출목적'] = test_df['대출목적'].replace({'이사':'주택', '주택개선':'주택'})
test_df.loc[~train_df['대출목적'].isin(['주택', '부채 통합', '신용 카드']), '대출목적'] = '기타'

## drop

In [59]:
train_df = train_df.drop(columns=['ID'])
test_df = test_df.drop(columns=['ID'])

## 라벨 인코딩

In [60]:
categorical_features = ['대출기간', '주택소유상태', '대출목적']

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_df[i]) 
    train_df[i]=le.transform(train_df[i])
    
    for case in np.unique(test_df[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test_df[i]=le.transform(test_df[i])

In [61]:
# target 피처 Label Encoding
target_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6} 
reverse_target_dict = {v: k for k, v in target_dict.items()} #submission을 위해 재변환을 위한 딕셔너리

In [62]:
# target 피처 Label Encoding
target_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6} 
reverse_target_dict = {v: k for k, v in target_dict.items()} #submission을 위해 재변환을 위한 딕셔너리
# apply 함수를 사용하여 Label Encoding 적용
train_df['대출등급'] = train_df['대출등급'].apply(lambda x: target_dict[x])

## train, test 데이터셋 

In [63]:
train_x = train_df.drop(columns=['대출등급'])
train_y = train_df['대출등급']

test_x = test_df.copy()

# CleanLab (이상 데이터 찾기)

## 숫자형 데이터 StandardScaler 적용 (knn을 위해)

In [64]:
numeric_features = ["대출금액", "연간소득", "부채_대비_소득_비율", "연간소득/총계좌수", "총상환원금비율", "총상환이자비율", "총연체금액", "연체계좌수/총계좌수", '연간소득/대출금액']
scaler = StandardScaler()
X_processed = train_x.copy()
X_processed[numeric_features] = scaler.fit_transform(train_x[numeric_features])

## 학습에 사용할 모델 정의 (xgboost)

In [65]:
clf = xgb.XGBClassifier()
num_crossval_folds = 5
pred_probs = cross_val_predict(
    clf,
    X_processed,
    train_y,
    cv=num_crossval_folds,
    method="predict_proba",
)

## knn 학습

In [66]:
KNN = NearestNeighbors(metric='euclidean', n_neighbors=5) # n_neighbors=5
KNN.fit(train_x.values)

knn_graph = KNN.kneighbors_graph(mode="distance")

## 이상데이터 찾기

In [67]:
data = {"X": X_processed.values, "y": train_y}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, knn_graph=knn_graph)

# lab_df = lab.get_issues()
# outlier_issue_index = lab_df[lab_df['is_outlier_issue']==True].index
# len(outlier_issue_index)
lab.report()

Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Error in non_iid: If a knn_graph is not provided, features must be provided to fit a new knn.
Failed to check for these issue types: [NonIIDIssueManager]

Audit complete. 12744 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
       outlier       12352
         label         198
near_duplicate         194

Dataset Information: num_examples: 96293, num_classes: 7


---------------------- outlier issues ----------------------

About this issue:
	Examples that are very different from the rest of the dataset 
    (i.e. potentially out-of-distribution or rare/anomalous instances).
    

Number of examples with this issue: 12352
Overall dataset quality in terms of this issue: 0.0851

Examples representing most severe instances of this issue:
       is_outlier_issue  outlier_score
73607              True 

- **outlier 이슈:** 12352개
- **label 이슈:** 198개
- **near_duplicate 이슈:** 194게

In [68]:
lab_df = lab.get_issues()
lab_df

,is_label_issue,label_score,is_outlier_issue,outlier_score,is_near_duplicate_issue,near_duplicate_score
0,False,0.284084,False,0.001146,False,0.999920
1,False,0.997005,False,0.126714,False,0.950431
2,False,0.996561,False,0.117219,False,0.895225
3,False,0.999403,False,0.100094,False,0.844929
4,False,0.992861,False,0.128138,False,0.951158
...,...,...,...,...,...,...
96288,False,0.998180,False,0.049453,False,0.985153
96289,False,0.995913,False,0.146508,False,0.841536
96290,False,0.998321,False,0.129524,False,0.938531
96291,False,0.996392,False,0.027733,False,0.996791


## 이상 데이터 drop

In [69]:
# 라벨 이슈, duplicate 이슈 데이터 인덱스 가져오기
label_issue_index = lab_df[lab_df['is_label_issue']==True].index

duplicate_issue_index = lab_df[lab_df['is_near_duplicate_issue']==True].index

In [70]:
# 이상 데이터 drop
train_x.drop(label_issue_index, inplace=True)
train_y.drop(label_issue_index, inplace=True)

train_x.drop(duplicate_issue_index, inplace=True)
train_y.drop(duplicate_issue_index, inplace=True)

train_x.reset_index(drop=True, inplace=True)
train_y.reset_index(drop=True, inplace=True)

# 학습 및 예측

In [72]:
## kfold 배깅 앙상블

# kfold 함수
def model_fitting(X_train, y_train, test, model, k):
    # k-Fold 설정
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # 각 fold의 모델로부터의 예측을 저장할 리스트와 f1 점수 리스트
    ensemble_predictions = []
    ensemble_predictions_train = []
    scores = []

    for train_idx, val_idx in tqdm(kf.split(X_train), total=k, desc="Processing folds"):
        X_t, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_t, y_val = y_train[train_idx], y_train[val_idx]
        
        # 각 모델 학습
        model.fit(X_t, y_t)
        
        # 각 모델로부터 Validation set에 대한 예측을 생성
        val_pred = model.predict(X_val)
        
        # Validation set에 대한 대회 평가 산식 계산 후 저장
        score = f1_score(y_val, val_pred, average='macro')
        scores.append(score)
        print(score)
        
        #train 데이터셋에 대해 앙상블 성능평가 (train 데이터셋에 대한 예측 수행 후 저장)
        model_pred_train = model.predict(train_x)
        ensemble_predictions_train.append(model_pred_train)    
        
        # test 데이터셋에 대한 예측 수행 후 저장
        model_pred = model.predict(test)        
        ensemble_predictions.append(model_pred)
        
    # K-fold 모든 예측의 평균을 계산하여 fold별 모델들의 voting 앙상블 예측 생성
    # test 데이터 앙상블
    final_predictions, _ = mode(ensemble_predictions, axis=0)
    final_predictions = final_predictions.ravel()
    
    # train 데이터 앙상블(성능평가용)
    final_predictions_train, _ = mode(ensemble_predictions_train, axis=0)
    final_predictions_train = final_predictions_train.ravel()
    
    # 각 fold에서의 Validation Metric Score와 전체 평균 Validation Metric Score출력
    print("Validation : fl scores for each fold:", scores)
    print("Validation : fl socres mean:", np.mean(scores))
    print("emsemble : fl socres train:", f1_score(train_y, final_predictions_train, average='macro'))
    return final_predictions

In [125]:
# knn 5개 + label, duplicate 이슈 drop
xgboost = xgb.XGBClassifier()
final_prediction = model_fitting(train_x, train_y, test_x, xgboost, 5)

Processing folds:  20%|██        | 1/5 [00:08<00:35,  8.90s/it]

0.9459334757584063


Processing folds:  40%|████      | 2/5 [00:18<00:27,  9.17s/it]

0.9559963723573709


Processing folds:  60%|██████    | 3/5 [00:27<00:18,  9.35s/it]

...

Processing folds:  80%|████████  | 4/5 [00:37<00:09,  9.52s/it]

0.952537755325867


Processing folds: 100%|██████████| 5/5 [00:47<00:00,  9.49s/it]

0.9479777548676349


Validation : fl scores for each fold: [0.9459334757584063, 0.9559963723573709, 0.9487197944760121, 0.952537755325867, 0.9479777548676349]
Validation : fl socres mean: 0.9502330305570583
emsemble : fl socres train: 0.9920976489469265


# 제출

In [161]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['대출등급'] = final_prediction
sample_submission['대출등급'] = sample_submission['대출등급'].apply(lambda x: reverse_target_dict[x])
sample_submission.to_csv('xgboost_7fold_cleanlab_label_duplicate_이슈drop.csv', index=False)